In [1]:
import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import re 

master_df = pd.read_csv("C:/Users/mgruz/Desktop/w210/data/nutrient/compiled/nutrition_master_df.csv")

with open('C:/Users/mgruz/Desktop/w210/data/recipe/NDB_NO_tag_dict.json') as f:
    NDB_NO_tag_dict = json.load(f)
    
with open('C:/Users/mgruz/Desktop/w210/data/recipe/recipe_all.json') as f:
    original_recipe = json.load(f)
    
with open('C:/Users/mgruz/Desktop/w210/data/recipe/recipe_clean.json') as f:
    recipe_clean = json.load(f)
    
with open('C:/Users/mgruz/Desktop/w210/data/recipe/recipe_clean_anna.json') as f:
    recipe_clean_anna = json.load(f)
    
with open('C:/Users/mgruz/Desktop/w210/data/recipe/recipe_clean_maura_v1.json') as f:
    recipe_clean_maura_v1 = json.load(f)

height has been deprecated.



In [2]:
for i in NDB_NO_tag_dict.keys():
    
    try:
        if '"' not in NDB_NO_tag_dict[i] and NDB_NO_tag_dict[i] != 'np.nan' and NDB_NO_tag_dict[i] != '':
            print i, NDB_NO_tag_dict[i]
            
            if '"' not in NDB_NO_tag_dict[i]:
                NDB_NO_tag_dict[i] = '"{}"'.format(NDB_NO_tag_dict[i].strip("\t"))
    except:
        print 'FAIL', i

duck fat  04542
ground all spice 02001
sweet dessert wine 14057
canned pumpkin 11424
peppercorns 02030
white bread dough 18014
dried red pepper flakes '02031'
boneless leg of lamb butterflied 17300
hot cherry peppers peppers
blanched whole almonds 12062
shredded red cabbage 11112
2-percent mozzarella cheese 01029
for purging exi
baby bok choy 11116
FAIL pkg martha white chocolate chip muffin mix
reduced-sodium chicken broth 06970
mini rigatoni 20420
dried hot red pepper 02031
raspberry 09302
bunch watercress stems 11591
english muffins 18639
five-spice powder 02031
fresh rosemary 02063
toasted pecans coarsely 12142
small-diced carrots 11124
dark raisins 09299
red radishes 11429


In [3]:
recipe_clean_ALL = {}

for key in recipe_clean.keys():
    recipe_clean_ALL[key] = recipe_clean[key]
    recipe_clean_ALL[key]['NDB_NO_tags'] = []
    
for key in recipe_clean_anna.keys():
    recipe_clean_ALL[key] = recipe_clean_anna[key]
    recipe_clean_ALL[key]['NDB_NO_tags'] = []
    
for key in recipe_clean_maura_v1.keys():
    recipe_clean_ALL[key] = recipe_clean_maura_v1[key]
    recipe_clean_ALL[key]['NDB_NO_tags'] = []

In [4]:
itr = 0

original_text = []
nbd_tag = []
for key in recipe_clean_ALL.keys():
    
    for i in recipe_clean_ALL[key]['ingredients']:
        
        ingredient_split = i.split(" ")
        ingredient_split = filter(None, ingredient_split)
        ingredient_new = " ".join(ingredient_split[2:])
        
        try:
            recipe_clean_ALL[key]['NDB_NO_tags'].append(NDB_NO_tag_dict[ingredient_new])
        except:
            recipe_clean_ALL[key]['NDB_NO_tags'].append('')
    
    
    if len(recipe_clean_ALL[key]['NDB_NO_tags']) == len(original_recipe[key]['ingredients']):       
        itr_2 = 0
        while itr_2 < len(recipe_clean_ALL[key]['NDB_NO_tags']):
            original_text.append(original_recipe[key]['ingredients'][itr_2])
            nbd_tag.append(recipe_clean_ALL[key]['NDB_NO_tags'][itr_2])
            itr_2 += 1
        
        itr += 1

In [5]:
units_of_food_recipe = ["cups", "cup", "c.", "c", 
                        "fl. oz.", "fl oz", "fluid ounce", "fluid ounces",
                        "gal", "gal.", "gallon", "gallons",
                        "oz", "oz.", "ounce", "ounces", "ouncs",
                        "pt", "pt.", "pint", "pints",
                        "lb", "lb.", "pound", "pounds",
                        "qt", "qt.", "qts", "qts.", "quart", "quarts",
                        "tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs",
                        "tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons",
                        "g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes",
                        "kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes",
                        "l", "l.", "liter", "liters", "litre", "litres",
                        "mg", "mg.", "milligram", "milligrams", "milligramme", "milligrammes",
                        "ml", "ml.", "milliliter", "milliliters", "millilitre", "millilitres",
                        "pinch", "pinches", "dash", "dashes", "touch", "touches", "handful", "handfuls",
                        "stick", "sticks",
                        "cans", "can",
                        "to taste",
                        "scoop", "scoops",
                        "dollop", "dollops",
                        "sprig", "sprigs",
                       "recipe",
                       "garnish", "garnished",
                       "sprinkle", "spinkled",
                       "slices",
                       "serving", "servings",
                       "ribs", "rib", "stalk",
                       "inch", "inches", "in.",
                       "drizzle", "drizzled",
                       "to taste"]
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [6]:
filtered_original_text = []
for original in original_text:
    original = re.sub(r'\s*(\d+|[./+*-])', '', original)
    original_split = original.split(" ")
    remove_list = list(set(original).symmetric_difference(units_of_food_recipe))
    
    for i in units_of_food_recipe:
        if i in original_split:
          original_split.remove(i)
    
    original_split = filter(None, original_split)
    new = " ".join(original_split)
    
    
    filtered_original_text.append(new)

In [8]:
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline


nlp_df = pd.DataFrame([original_text, filtered_original_text, nbd_tag]).transpose()
nlp_df.columns = ['original_text', 'filtered_text','NBD_tag']
nlp_df = nlp_df[nlp_df['NBD_tag'] != 'np.nan']
nlp_df['NBD_tag'].replace('', np.nan, inplace=True)
nlp_df.dropna(subset=['NBD_tag'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(nlp_df['filtered_text'], nlp_df['NBD_tag'], random_state=2, train_size=0.7)
X_train_use, X_dev, y_train_use, y_dev = train_test_split(X_train, y_train, random_state=2, train_size=0.8)


vectorizer = CountVectorizer()
vecorized_train = vectorizer.fit_transform(X_train_use)
vecorized_test = vectorizer.transform(X_test)
vecorized_dev = vectorizer.transform(X_dev)

C:\Users\mgruz\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\mgruz\Anaconda2\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\mgruz\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
knn_values = []
knn_results = []
for i in range(1, 100):
    knn_f1 = KNeighborsClassifier(n_neighbors=i)
    knn_f1.fit(vecorized_train, y_train_use)
    knn_f1_prediction = knn_f1.predict(vecorized_dev)
    knn_f1_metric = metrics.f1_score(y_dev, knn_f1_prediction, average='micro') * 100
    knn_values.append(i)
    knn_results.append(knn_f1_metric)
    
fig = plt.figure()
plt.plot(knn_values, knn_results)

In [ ]:
mnb_values = []
mnb_results = []
for i in [0.000001, 0.00001, 0.0001, 0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]:
    mnb_f1 = MultinomialNB(alpha=i)
    mnb_f1.fit(vecorized_train, y_train_use)
    mnb_f1_prediction = mnb_f1.predict(vecorized_dev)
    mnb_f1_metric = metrics.f1_score(y_dev, mnb_f1_prediction, average='micro') * 100
    mnb_values.append(i)
    mnb_results.append(mnb_f1_metric)

fig = plt.figure()
plt.plot(mnb_values, mnb_results)

In [ ]:
lr_values = []
lr_results = []    
for i in [0.000001, 0.00001, 0.0001, 0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10, 20]:
    lr_f1 = LogisticRegression(penalty='l2', C=i)
    lr_f1.fit(vecorized_train, y_train_use)
    lr_f1_prediction = lr_f1.predict(vecorized_dev)
    lr_f1_metric = metrics.f1_score(y_dev, lr_f1_prediction, average='micro')  * 100
    lr_values.append(i)
    lr_results.append(lr_f1_metric)
    
fig = plt.figure()
plt.plot(lr_values, lr_results)

In [ ]:
rf_values = []
rf_results = []  
for i in range(1, 500, 50):
    rf_f1 = RandomForestClassifier(n_estimators=i)
    rf_f1.fit(vecorized_train, y_train_use)
    rf_f1_prediction = rf_f1.predict(vecorized_dev)
    rf_f1_metric = metrics.f1_score(y_dev, rf_f1_prediction, average='micro')  * 100
    rf_values.append(i)
    rf_results.append(rf_f1_metric)
plt.plot(rf_values, rf_results)

rf_f1 = RandomForestClassifier(n_estimators=50)
rf_f1.fit(vecorized_train, y_train_use)
rf_f1_prediction = rf_f1.predict(vecorized_dev)
rf_f1_metric = metrics.f1_score(y_dev, rf_f1_prediction, average='micro')  * 100
print 'rf_f1_metric', rf_f1_metric

In [ ]:
dt_f1 = DecisionTreeClassifier()
dt_f1.fit(vecorized_train, y_train_use)
dt_f1_prediction = dt_f1.predict(vecorized_dev)
dt_f1_metric = metrics.f1_score(y_dev, dt_f1_prediction, average='micro')  * 100
print 'dt_f1_metric', dt_f1_metric

In [ ]:
ab_fit = AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=600)
ab_fit.fit(vecorized_train, y_train_use)
ab_fit_prediction = ab_fit.predict(vecorized_dev)
ab_fit_metric = metrics.f1_score(y_dev, ab_fit_prediction, average='micro')  * 100
print 'ab_fit_metric', ab_fit_metric

In [ ]:
et_values = []
et_results = []  
for i in range(1, 500, 50):
    et_f1 = RandomForestClassifier(n_estimators=i)
    et_f1.fit(vecorized_train, y_train_use)
    et_f1_prediction = et_f1.predict(vecorized_dev)
    et_f1_metric = metrics.f1_score(y_dev, et_f1_prediction, average='micro')  * 100
    et_values.append(i)
    et_results.append(et_f1_metric)
plt.plot(et_values, et_results)

et_f1 = RandomForestClassifier(n_estimators=100)
et_f1.fit(vecorized_train, y_train_use)
et_f1_prediction = et_f1.predict(vecorized_dev)
et_f1_metric = metrics.f1_score(y_dev, et_f1_prediction, average='micro')  * 100
print 'et_f1_metric', et_f1_metric

In [ ]:
gb_fit = GradientBoostingClassifier()
gb_fit.fit(vecorized_train.todense(), y_train_use)
gb_fit_prediction = gb_fit.predict(vecorized_dev.todense())
gb_fit_metric = metrics.f1_score(y_dev, gb_fit_prediction, average='micro')  * 100
print 'gb_fit_metric', gb_fit_metric

In [ ]:
lr_f1_prediction = lr_f1.predict(vecorized_test)
lr_f1_metric = metrics.f1_score(y_test, lr_f1_prediction, average='micro')  * 100
print 'lr_f1_metric', lr_f1_metric

In [ ]:
rf_f1_prediction = rf_f1.predict(vecorized_test)
rf_f1_metric = metrics.f1_score(y_test, rf_f1_prediction, average='micro')  * 100
print 'rf_f1_metric', rf_f1_metric

In [ ]:
dt_f1_prediction = dt_f1.predict(vecorized_test)
dt_f1_metric = metrics.f1_score(y_test, dt_f1_prediction, average='micro')  * 100
print 'dt_f1_metric', dt_f1_metric

In [ ]:
et_f1_prediction = et_f1.predict(vecorized_test)
et_f1_metric = metrics.f1_score(y_test, et_f1_prediction, average='micro')  * 100
print 'et_f1_metric', et_f1_metric

In [ ]:
ab_fit_prediction = ab_fit.predict(vecorized_test)
ab_fit_metric = metrics.f1_score(y_test, ab_fit_prediction, average='micro')  * 100
print 'ab_fit_metric', ab_fit_metric

In [ ]:
gb_fit_prediction = gb_fit.predict(vecorized_test.todense())
gb_fit_metric = metrics.f1_score(y_test, gb_fit_prediction, average='micro')  * 100
print 'gb_fit_metric', gb_fit_metric

In [ ]:
from scipy.stats import mode


output_df = pd.DataFrame([lr_f1_prediction, rf_f1_prediction, dt_f1_prediction, ab_fit_prediction, et_f1_prediction, y_test]).transpose()
output_df.columns = ["lr_f1_prediction", "rf_f1_prediction", "dt_f1_prediction", "ab_fit_prediction", "et_f1_prediction", "y_test"]
output_df['Description'] = ''
output_df['highest_mode'] = ''
output_df['Description_highest_mode'] = ''
output_df['text'] = ''

itr = 0
while itr < len(output_df):
    
    temp_mode = output_df.loc[itr, ["lr_f1_prediction", "rf_f1_prediction", "dt_f1_prediction", "ab_fit_prediction", "et_f1_prediction"]].mode()
    temp_mode = temp_mode.get_values()[0]
        
    output_df.loc[itr, "highest_mode"] = temp_mode
    output_df.loc[itr, "y_test"] = output_df.loc[itr, "y_test"].strip("]")
    try:
        output_df.loc[itr, 'Description'] = master_df[master_df['NDB_NO'] == "\"{}\"".format(output_df.loc[itr, 'y_test'].strip('"'))]['Description'].get_values()[0]
    except:
        print itr,"FAILED"
    
    
    output_df.loc[itr, 'Description_highest_mode'] = master_df[master_df['NDB_NO'] == "\"{}\"".format(temp_mode.strip('"'))]['Description'].get_values()[0]
    
#     print itr, X_test.iloc[itr]
    
    output_df.loc[itr, 'text'] = X_test.iloc[itr]
    
    itr += 1
    
    


# output_df.to_csv("C:/Users/mgruz/Desktop/w210/data/recipe/check/output_prediction_ml.csv")